In [5]:
import spacy
import pandas as pd

nlp = spacy.blank("en")

df1=pd.read_csv("./1/twitter_training.csv")

#dataset1 = list(df1[["text", "value"]].sample(frac=1).itertuples(index=False, name=None))
dataset1=df1[["value","text"]]

In [6]:
df2=pd.read_csv("./2/trainingdata.csv", encoding="iso-8859-1")
df2.columns = ["value","somenumber", "date", "noquery","user","text"]

#trainingdata2=df2[["value","text"]]
dataset2=df2[["value","text"]]

#dataset2 = list(trainingdata2[["text", "value"]].sample(frac=1).itertuples(index=False, name=None))


In [7]:
filtered_dataset1 = dataset1[dataset1['value'].str.isnumeric()]

filtered_dataset2=dataset2
# Convert 'value' column to integers
filtered_dataset1.loc[:, 'value'] = filtered_dataset1['value'].astype(int)
filtered_dataset2.loc[:, 'value'] = filtered_dataset2['value'].astype(int)

# Filter rows with 'value' equal to 0, 2, or 4
filtered_dataset1 = filtered_dataset1[filtered_dataset1['value'].isin([0, 2, 4])]
filtered_dataset2 = filtered_dataset2[filtered_dataset2['value'].isin([0, 2, 4])]

# Create datasets with selected columns
train_dataset1 = list(filtered_dataset1[["text", "value"]].sample(frac=1).itertuples(index=False, name=None))
train_dataset2 = list(filtered_dataset2[["text", "value"]].sample(frac=1).itertuples(index=False, name=None))


In [8]:
combined_df=train_dataset1+train_dataset2

#combined_df = pd.concat([modset1, modset2], axis=0)

columns = [ 'text','value']

training_df = pd.DataFrame(combined_df,columns=columns)

training_dataset = list(training_df[["text", "value"]].sample(frac=1).itertuples(index=False, name=None))

#print(len(training_dataset))

#1674672 = 74673 + 1599998

In [9]:
import re

train_doc_data=[]
for text, value in training_dataset:
    if isinstance(text, str) and isinstance(value, int):
        cleaned_text = re.sub(r'[^A-Za-z0-9\s]', '', text)    
        doc = nlp(cleaned_text)
        train_doc_data.append((doc, value))

#7sec (en_web)=1000
#159sec (en)=1.7million

In [10]:
df3=pd.read_csv("./1/twitter_validation.csv")
df4=pd.read_csv("./2/testdata.csv", encoding="iso-8859-1")

df3.columns = ["random","org", "value", "text"]
#df4.columns = ["value","somenumber", "date", "noquery","user","text"]

trainingdata3=df3[["value","text"]]

df4.columns = ["value","somenumber", "date", "noquery","user","text"]

trainingdata4=df4[["value","text"]]


In [11]:
filtered_dataset3 = trainingdata3[trainingdata3['value'].str.isnumeric()]
filtered_dataset4=trainingdata4
# Convert 'value' column to integers
filtered_dataset3.loc[:, 'value'] = filtered_dataset3['value'].astype(int)
filtered_dataset4['value'] = filtered_dataset4['value'].astype(int)

# Filter rows with 'value' equal to 0, 2, or 4
filtered_dataset3 = filtered_dataset3[filtered_dataset3['value'].isin([0, 2, 4])]
filtered_dataset4 = filtered_dataset4[filtered_dataset4['value'].isin([0, 2, 4])]

# Create datasets with selected columns
dataset3 = list(filtered_dataset3[["text", "value"]].sample(frac=1).itertuples(index=False, name=None))
dataset4 = list(filtered_dataset4[["text", "value"]].sample(frac=1).itertuples(index=False, name=None))

# Combine the two datasets
#test_combined_df = dataset3 + dataset4

C:\Users\Abdur\AppData\Local\Temp\ipykernel_3824\411516550.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_dataset4['value'] = filtered_dataset4['value'].astype(int)


In [12]:
#dataset3 = list(trainingdata3[["text", "value"]].sample(frac=1).itertuples(index=False, name=None))
#dataset4 = list(trainingdata4[["text", "value"]].sample(frac=1).itertuples(index=False, name=None))

test_combined_df=dataset3+dataset4

#print(combined_df)


In [13]:
columns = [ 'text','value']
testing_df = pd.DataFrame(test_combined_df,columns=columns)


In [14]:
testing_dataset = list(testing_df[["text", "value"]].sample(frac=1).itertuples(index=False, name=None))

In [15]:
testing_doc_data = []
for text, value in testing_dataset:
    if isinstance(text, str) and isinstance(value, int):
        cleaned_test_text = re.sub(r'[^A-Za-z0-9\s]', '', text)    
        doc = nlp(cleaned_test_text)
        testing_doc_data.append((doc, value))


In [16]:
import string
import re
import spacy
from spacy.lang.pt.stop_words import STOP_WORDS

def preprocessing(text):
  text = text.lower()
  
  tokens = [token.text for token in nlp(text)]
  
  tokens = [t for t in tokens if 
              t not in STOP_WORDS and 
              t not in string.punctuation and 
              len(t) > 3]
  
  tokens = [t for t in tokens if not t.isdigit()]
    
  return " ".join(tokens)

In [21]:
def convert(data, outfile):
    db = spacy.tokens.DocBin()
    docs = []
    for doc, value in nlp.pipe(data, as_tuples=True):
        doc.cats["Negative"] = value == 0
        doc.cats["Neutral"] = value == 2
        doc.cats["Positive"] = value == 4
        db.add(doc)
    
    db.to_disk(outfile)

#print(len(train_doc_data[:830560]))
#print(len(testing_doc_data))
convert(train_doc_data[:830560], "./train.spacy")
convert(testing_doc_data, "./dev.spacy")

In [23]:
!python -m spacy init config  --lang pt --pipeline textcat --optimize efficiency --force config.cfg

[!] To generate a more effective transformer-based config (GPU-only),
install the spacy-transformers package and re-run this command. The config
generated now does not use transformers.
[i] Generated config template specific for your use case
- Language: pt
- Pipeline: textcat
- Optimize for: efficiency
- Hardware: CPU
- Transformer: None
[+] Auto-filled config with all values
[+] Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [24]:
!python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy

^C
